In [1]:
import numpy as np
from scipy import sparse
from scipy.sparse import csr_matrix

import time
import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [2]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [4]:
%load_ext autoreload
%autoreload 2

from pubmed_landscape_src.metrics import knn_accuracy, knn_accuracy_years, chance_knn_accuracy, knn_accuracy_years_chance

In [3]:
variables_path = Path('../results/variables')
figures_path = Path('../results/figures')
berenslab_data_path = Path('/gpfs01/berens/data/data/pubmed_processed')

In [4]:
%load_ext autoreload
%autoreload 2
import pubmed_landscape_src.metrics

In [6]:
pubmed_landscape_src.metrics?

Type:        module
String form: <module 'pubmed_landscape_src.metrics' from '/gpfs01/berens/user/rgonzalesmarquez/phd/pubmed-landscape/pubmed_landscape_src/metrics.py'>
File:        ~/phd/pubmed-landscape/pubmed_landscape_src/metrics.py
Docstring:  
Hi
Hello


# Import data

## TF-IDF

In [ ]:
tfidf_features_updated_reparsed = sp.sparse.load_npz(variables_path / "tfidf_features_updated_reparsed.npz")

tcmalloc: large alloc 18038185984 bytes == 0x28de994000 @ 


In [2]:
svd_data_updated_reparsed = np.load(variables_path / "svd_data_updated_reparsed.npy")

tcmalloc: large alloc 49649164288 bytes == 0x5900000 @ 


In [105]:
tsne_tfidf_reparsed=np.load(variables_path / "tsne_tfidf_reparsed.npy")

## BERT

In [83]:
%%time
%%memit
embeddings_reparsed_all=np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_all.npy')

tcmalloc: large alloc 127101853696 bytes == 0x439a2000 @ 


peak memory: 122636.70 MiB, increment: 117918.22 MiB
CPU times: user 271 ms, sys: 1min 3s, total: 1min 4s
Wall time: 1min 43s


In [6]:
tsne_bert_reparsed=np.load(variables_path / "bert/tsne_reparsed.npy")

## Labels

In [82]:
# new - should actually give the same result as colors_new
colors_new = np.load(variables_path / "colors_new.npy", allow_pickle = True)

In [45]:
date_year = np.load(variables_path / "date_year_reparsed.npy")

# kNN accuracies

## low-dim space

In [8]:
knn_accuracy_tfidf_bert_tsne_reparsed_new_colors= knn_accuracy([tsne_tfidf_reparsed, tsne_bert_reparsed] , colors_new, subset_size=10000)

In [9]:
print(knn_accuracy_tfidf_bert_tsne_reparsed_new_colors)

[0.5064, 0.6261]


In [ ]:
np.save(variables_path / "knn_accuracy_tfidf_bert_tsne_reparsed_new_colors",knn_accuracy_tfidf_bert_tsne_reparsed_new_colors)

In [2]:
#knn_accuracy_tfidf_bert_tsne_reparsed_new_colors=np.load(variables_path / "knn_accuracy_tfidf_bert_tsne_reparsed_new_colors.npy")

## mid-dim space

### SVD

In [9]:
%%time
%%memit
knn_accuracy_svd_reparsed_new_colors= knn_accuracy([svd_data_updated_reparsed] , colors_new, subset_size=500)

tcmalloc: large alloc 16724328448 bytes == 0x120e876000 @ 
tcmalloc: large alloc 16723132416 bytes == 0x15f4606000 @ 
tcmalloc: large alloc 16724328448 bytes == 0x120e876000 @ 


peak memory: 82492.72 MiB, increment: 8371.35 MiB
CPU times: user 17min 51s, sys: 2min 42s, total: 20min 33s
Wall time: 3min 38s


In [10]:
print(knn_accuracy_svd_reparsed_new_colors)

[0.6]


In [11]:
np.save(variables_path / "knn_accuracy_svd_reparsed_new_colors", knn_accuracy_svd_reparsed_new_colors)

## high-dim space

### BERT

In [84]:
%%time
%%memit
knn_accuracy_bert_n_5000 = knn_accuracy([embeddings_reparsed_all] , colors_new, subset_size=5000)

tcmalloc: large alloc 43768053760 bytes == 0x1eab51c000 @ 
tcmalloc: large alloc 43737333760 bytes == 0x28de994000 @ 
tcmalloc: large alloc 43768053760 bytes == 0x1eab51c000 @ 


peak memory: 209513.21 MiB, increment: 86876.51 MiB
CPU times: user 7h 56min 6s, sys: 2h 53min 52s, total: 10h 49min 59s
Wall time: 1h 28min 48s


In [85]:
print(knn_accuracy_bert_n_5000)

[0.697]


In [86]:
np.save(variables_path / "knn_accuracy_bert_n_5000", knn_accuracy_bert_n_5000)

### TF-IDF

In [ ]:
%%time
knn_accuracy_tfidf_n_5000 = knn_accuracy([tfidf_features_updated_reparsed] , colors_new, subset_size=5000)

In [114]:
print(knn_accuracy_tfidf_n_5000)

[0.6522]


In [ ]:
np.save(variables_path / "knn_accuracy_tfidf_n_5000", knn_accuracy_tfidf_n_5000)

## Chance

In [18]:
%%time
%%memit
knn_accuracy_chance = chance_knn_accuracy([tsne_bert_reparsed] , colors_new, subset_size=10000)

peak memory: 1256.66 MiB, increment: 0.16 MiB
CPU times: user 8.04 s, sys: 463 ms, total: 8.51 s
Wall time: 8.67 s


In [19]:
print(knn_accuracy_chance)

[0.0431]


In [20]:
np.save(variables_path / "knn_accuracy_chance",knn_accuracy_chance)

# Root-mean-squared error of kNN prediction of publication year

## low-dim space

### t-SNE BERT

In [90]:
%%time
knn_accuracy_years_bert_tsne = knn_accuracy_years([tsne_bert_reparsed], date_year, k=10, subset_size=10000)

CPU times: user 2h 30min 59s, sys: 1h 58min 28s, total: 4h 29min 28s
Wall time: 56min 52s


In [104]:
knn_accuracy_years_bert_tsne
np.sqrt(knn_accuracy_years_bert_tsne)

array([10.22518753])

In [94]:
np.save(variables_path / "knn_accuracy_years_bert_tsne", knn_accuracy_years_bert_tsne)

### t-SNE TF-IDF

In [106]:
%%time
knn_accuracy_years_tfidf_tsne = knn_accuracy_years([tsne_tfidf_reparsed], date_year, k=10, subset_size=10000)

CPU times: user 2h 8min 4s, sys: 1h 56min 29s, total: 4h 4min 34s
Wall time: 55min 2s


In [107]:
knn_accuracy_years_tfidf_tsne
np.sqrt(knn_accuracy_years_tfidf_tsne)

array([11.16393528])

In [108]:
np.save(variables_path / "knn_accuracy_years_tfidf_tsne", knn_accuracy_years_tfidf_tsne)

## high-dim space

### BERT

In [87]:
%%time
knn_accuracy_years_bert = knn_accuracy_years([embeddings_reparsed_all], date_year, subset_size=5000)

tcmalloc: large alloc 127071133696 bytes == 0x33100c0000 @ 


CPU times: user 2d 1h 29min 52s, sys: 6h 8min 38s, total: 2d 7h 38min 31s
Wall time: 8h 17min 8s


In [103]:
knn_accuracy_years_bert
np.sqrt(knn_accuracy_years_bert)

array([8.4263444])

In [89]:
np.save(variables_path / "knn_accuracy_years_bert", knn_accuracy_years_bert)

### TF-IDF

In [87]:
%%time
knn_accuracy_years_tfidf = knn_accuracy_years([tfidf_features_updated_reparsed], date_year, subset_size=5000)

tcmalloc: large alloc 127071133696 bytes == 0x33100c0000 @ 


CPU times: user 2d 1h 29min 52s, sys: 6h 8min 38s, total: 2d 7h 38min 31s
Wall time: 8h 17min 8s


In [101]:
knn_accuracy_years_tfidf
np.sqrt(knn_accuracy_years_tfidf)

array([8.84692749])

In [89]:
np.save(variables_path / "knn_accuracy_years_tfidf", knn_accuracy_years_tfidf)

## Chance

In [ ]:
%%capture cap
%%time
knn_accuracy_years_chance_bert = knn_accuracy_years_chance([tsne_bert_reparsed], date_year, k=10, subset_size=5000, rs=42)

In [ ]:
with open( variables_path / 'verbose_knn_accuracy_years_chance_bert.txt', 'w') as f:
    f.write(cap.stdout)

In [18]:
print(np.sqrt(knn_accuracy_years_chance_bert))

[12.44069974]


In [ ]:
np.save(variables_path / "knn_accuracy_years_chance_bert", knn_accuracy_years_chance_bert)